In [1]:
import math
import unidecode

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tldextract
import urllib.parse

from foursquare.dataset import extract_features_url

pd.set_option('display.max_colwidth', None)

2022-08-06 20:43:09.137708: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-06 20:43:09.137726: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/pablo/miniconda3/envs/foursquare/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


imported sklearn.neighbors.NearestNeighborsDefault


In [2]:
def descriptive(series):
    counts = series.value_counts()
    print(counts.head(10))
    rows_common = counts[counts >= 5].sum()
    classes_common = counts[counts >= 5].count()
    print("not null", series.notnull().sum())
    print("rows (common classes)", rows_common, "classes", classes_common)
    print(counts.describe(percentiles=[0.9, 0.95, 0.99, 0.999]))

In [3]:
df = pd.read_csv("/kaggle/input/foursquare-location-matching/train.csv")
print(df.shape)
print(df.isnull().sum(axis=0))
df.head()

(1138812, 13)
id                        0
name                      1
latitude                  0
longitude                 0
address              396621
city                 299189
state                420586
zip                  595426
country                  11
url                  871088
phone                795957
categories            98307
point_of_interest         0
dtype: int64


,id,name,latitude,longitude,address,city,state,zip,country,url,phone,categories,point_of_interest
0,E_000001272c6c5d,Café Stad Oudenaarde,50.859975,3.634196,Abdijstraat,Nederename,Oost-Vlaanderen,9700,BE,NaN,NaN,Bars,P_677e840bb6fc7e
1,E_000002eae2a589,Carioca Manero,-22.907225,-43.178244,NaN,NaN,NaN,NaN,BR,NaN,NaN,Brazilian Restaurants,P_d82910d8382a83
2,E_000007f24ebc95,ร้านตัดผมการาเกด,13.780813,100.484900,NaN,NaN,NaN,NaN,TH,NaN,NaN,Salons / Barbershops,P_b1066599e78477
3,E_000008a8ba4f48,Turkcell,37.844510,27.844202,Adnan Menderes Bulvarı,NaN,NaN,NaN,TR,NaN,NaN,Mobile Phone Shops,P_b2ed86905a4cd3
4,E_00001d92066153,Restaurante Casa Cofiño,43.338196,-4.326821,NaN,Caviedes,Cantabria,NaN,ES,NaN,NaN,Spanish Restaurants,P_809a884d4407fb


In [4]:
df_url = extract_features_url(df[["url"]])
df_url.dropna(how="all").head()

,url_scheme,netloc,path,params,query,fragment,subdomain,url_domain,suffix,url_netloc,url_path,url_params,url_query,url_fragment,url_subdomain,url_suffix
8,https,NaN,NaN,NaN,NaN,NaN,NaN,togos,NaN,locations.togos.com,/ll/US/CA/Fairfield/1380-Holiday-Ln_*-Ste_-B,NaN,NaN,NaN,locations,com
12,https,NaN,NaN,NaN,NaN,NaN,NaN,tsukemen-tsujita,NaN,tsukemen-tsujita.com,NaN,NaN,NaN,NaN,NaN,com
13,http,NaN,NaN,NaN,NaN,NaN,NaN,oursignatureproperties,NaN,www.oursignatureproperties.com,NaN,NaN,NaN,NaN,www,com
16,http,NaN,NaN,NaN,NaN,NaN,NaN,innerworkingsmassage,NaN,innerworkingsmassage.com,NaN,NaN,NaN,NaN,NaN,com
17,http,NaN,NaN,NaN,NaN,NaN,NaN,lawson,NaN,www.lawson.co.jp,NaN,NaN,NaN,NaN,www,co.jp


In [5]:
descriptive(df_url["url_scheme"])

http     183351
https     84237
Name: url_scheme, dtype: int64
not null 267588
rows (common classes) 267588 classes 2
count         2.000000
mean     133794.000000
std       70084.181511
min       84237.000000
50%      133794.000000
90%      173439.600000
95%      178395.300000
99%      182359.860000
99.9%    183251.886000
max      183351.000000
Name: url_scheme, dtype: float64


In [6]:
descriptive(df_url["url_netloc"])

www.redbox.com            3146
www.walmart.com           2666
www.starbucks.com         1550
www.sej.co.jp             1218
restaurants.subway.com    1099
payless.com               1026
www.family.co.jp           985
www.marriott.com           935
www.payless.com            911
www.7eleven.co.th          882
Name: url_netloc, dtype: int64
not null 267567
rows (common classes) 139321 classes 5805
count    109086.000000
mean          2.452808
std          19.214608
min           1.000000
50%           1.000000
90%           2.000000
95%           5.000000
99%          24.000000
99.9%       154.000000
max        3146.000000
Name: url_netloc, dtype: float64


In [7]:
descriptive(df_url["url_path"])

/                           8616
/find-location.htm           602
/en                          361
/offices                     358
/ontheroad                   280
/maps/store-locator.html     250
/mystore/index.jsp           245
/en/                         200
/parks                       188
/index.html                  185
Name: url_path, dtype: int64
not null 91820
rows (common classes) 20954 classes 574
count    64338.000000
mean         1.427150
std         34.282056
min          1.000000
50%          1.000000
90%          2.000000
95%          2.000000
99%          4.000000
99.9%       36.663000
max       8616.000000
Name: url_path, dtype: float64


In [8]:
descriptive(df_url["url_params"])

146857522;s                                            10
jsessionid=00D28E211439525E2403DCAFC2A1EEC2.tomcat1     1
jsessionid=D2F64E9353542B65CFEE86915944DDDA             1
145636969;l;u=Y_RAPHTHT_W_D                             1
124607339;i                                             1
251241745;i                                             1
145636969;l;u=Y_EWRFJDT_W_D                             1
jsessionid=h0utysxzy4sk2csgbjnmvcq                      1
Name: url_params, dtype: int64
not null 17
rows (common classes) 10 classes 1
count     8.000000
mean      2.125000
std       3.181981
min       1.000000
50%       1.000000
90%       3.700000
95%       6.850000
99%       9.370000
99.9%     9.937000
max      10.000000
Name: url_params, dtype: float64


In [9]:
descriptive(df_url["url_query"])

jp_cmp=rb/LocalListings/aff/branch/na                               247
mc=llyxyxpx                                                         192
iata=00171890&ssob=BLBWI0003G                                       105
redirect=homepage                                                    90
loc=ns100                                                            78
extid=local                                                          76
PID=PSLocalSearch&CHID=LL                                            76
pk_campaign=Corporate&pk_source=WebClick&pk_medium=local_listing     67
facilitatorId=GLOBALSEO                                              46
corporatePlusNumber=CP792N5W&travelAgentNumber=TA001305              46
Name: url_query, dtype: int64
not null 13224
rows (common classes) 1704 classes 62
count    11107.000000
mean         1.190601
std          3.737486
min          1.000000
50%          1.000000
90%          1.000000
95%          1.000000
99%          2.000000
99.9%       45.000000

In [10]:
descriptive(df_url["url_fragment"])

utm_source=rest_of_network&utm_medium=Yext&utm_campaign=Listings    30
development=1                                                       30
utm_source=yext&utm_medium=localsearch&utm_campaign=yext            19
xtor=CS2-21-[uex]-[]-[]-[]                                          13
/                                                                   10
Vitoria                                                              5
storedetails                                                         5
details                                                              4
!                                                                    3
/es/es                                                               3
Name: url_fragment, dtype: int64
not null 759
rows (common classes) 112 classes 7
count    627.000000
mean       1.210526
std        1.906861
min        1.000000
50%        1.000000
90%        1.000000
95%        1.000000
99%        4.740000
99.9%     30.000000
max       30.000000
Name: ur

In [11]:
descriptive(df_url["url_subdomain"])

www            192504
locations        3798
stores           3075
restaurants      1385
tools             602
local             559
find              398
shop              364
locator           284
all               281
Name: url_subdomain, dtype: int64
not null 220126
rows (common classes) 212747 classes 385
count      6406.000000
mean         34.362473
std        2406.013508
min           1.000000
50%           1.000000
90%           3.000000
95%           6.000000
99%          45.900000
99.9%       384.230000
max      192504.000000
Name: url_subdomain, dtype: float64


In [12]:
descriptive(df_url["url_domain"])

starbucks    4304
redbox       3151
walmart      2848
mcdonalds    2654
subway       2078
payless      1939
sej          1218
shell        1119
kfc          1110
family       1014
Name: url_domain, dtype: int64
not null 267561
rows (common classes) 153599 classes 5513
count    94719.000000
mean         2.824787
std         28.404551
min          1.000000
50%          1.000000
90%          3.000000
95%          5.000000
99%         28.000000
99.9%      188.282000
max       4304.000000
Name: url_domain, dtype: float64


In [13]:
descriptive(df_url["url_suffix"])

com       135671
co.jp       9455
ru          9374
com.br      7492
de          7407
co.uk       6899
be          6718
com.tr      5833
org         5394
jp          4445
Name: url_suffix, dtype: int64
not null 266883
rows (common classes) 265893 classes 371
count       942.000000
mean        283.315287
std        4479.632175
min           1.000000
50%           3.000000
90%         118.900000
95%         443.950000
99%        4103.060000
99.9%     16901.744000
max      135671.000000
Name: url_suffix, dtype: float64


In [14]:
df["url"].dropna().apply(len)

8          71
12         28
13         37
16         31
17         23
           ..
1138781    14
1138784    22
1138788    24
1138797    94
1138806    80
Name: url, Length: 267724, dtype: int64